In [35]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup

In [73]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd

# Setup Selenium WebDriver
service = Service(
    r"C:\Users\Nishant shah\OneDrive\Desktop\Nishant\Software\chromedriver.exe"
)
driver = webdriver.Chrome(service=service)

# URL of the web page to scrape
url = "https://www.broadarrowauctions.com/vehicles/results?q%5Bbranch_id_eq%5D=15&q%5Bs%5D%5B0%5D%5Bname_dir%5D=stock.asc"
driver.get(url)

# Parse the page source with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")
vehicles = soup.find_all("div", class_="ft-vehicle-item grid-block")

data = []

# Extract and print vehicle details
for vehicle in vehicles:
    # Extract car name (combining all parts)
    car_name_section = vehicle.find("div", class_="name-auto")
    if car_name_section:
        car_name = " ".join(
            part.strip() for part in car_name_section.stripped_strings
        )  # Combine all strings into a single name
    else:
        car_name = "N/A"

    # Extract lot and auction details
    top_section = vehicle.find("div", class_="price")
    if top_section:

        spans = top_section.find_all("span")
        lot = (
            spans[0].text.replace("Lot", "").replace(" |", "").strip()
            if len(spans) > 0
            else "N/A"
        )
        auction_name = spans[1].text.strip() if len(spans) > 1 else "N/A"
    else:
        lot = "N/A"
        auction_name = "N/A"

    # Extract sold price
    sold_price_section = vehicle.find("div", class_="bot")
    sold_price = (
        sold_price_section.text.replace("Sold Price: ", "").strip()
        if sold_price_section
        else "N/A"
    )

    data.append([car_name, lot, auction_name, sold_price])

# Convert the data into a DataFrame
df = pd.DataFrame(data, columns=["Car Name", "Lot", "Auction Name", "Sold Price"])

# Save the DataFrame to an Excel file
df.to_excel("auction_data.xlsx", index=False, engine="openpyxl")

# Print message indicating that the data has been saved
print("Data has been saved to 'auction_data.xlsx'.")

# Close the browser
driver.quit()

Data has been saved to 'auction_data.xlsx'.
